# 随机森林和AdaBoost

#1 弱分类器和强分类器
最好的分类器是支持向量机,通常正确率为50%左右的是比较若的分类器，而正确率在80%或者90%以上的就是比较好的分类器。我们可以联合使用多个弱分类器，来形成一个强的分类器

#2 分类器的组合算法
- 装袋(bagging)
- 提升(boosting) - Adaboost
- 随机森林

我们可以用多个弱分类器，然后通过这三种不同的框架来实现一个强的分类器

#3 装袋算法和随机森林

##3.1 自助式抽样
我们需要构造多个弱分类器，但是又不能多次重复使用统一分数据(因为这样做只会得到同一个结果)，所以就需要构造不同的数据。用来构造不同的数据集的方法，就是自助式抽样

从总体m条记录中，有放回的抽出m个样本，组成的就是自助式抽样，由于是有放回的，所以抽出的m条记录是可能有重复记录的，所以两个m是不一样的。再一次抽样过程中，某一个样本被抽中的概率是$\frac{1}{m}$，那么它不被抽中的概率就是$1-\frac{1}{m}$，那么在一轮自助式抽样过程中，这个样本都没有被抽中的概率是$p=(1-\frac{1}{m})^m$，那么我们极限$\lim\limits_{m\to\infty}p=\lim\limits_{m\to\infty}(1-\frac{1}{m})^m=e^{-1}\approx0.368$，所以用自助式抽样所抽出的样本，能够覆盖总体样本的$1-0.368=0.632=63.2$%

##3.2 装袋算法
Bagging(Bootstrap aggregating的缩写)算法是最早的集成学习算法，具体步骤可以描述为：
- 利用Bootstrap方法重采样(自助式抽样)，随机产生T个训练集$S_1,S_2,...,S_T$;
- 利用每个训练集，生成对应的决策树$C_1,C_2,...,C_T$；
- 对于测试数据集样本X，利用每个决策树进行测试，得到对应的类别$C_1(X),C_2(X),...,C_T(X)$；
- 采用投票的方法，将T个决策树中输出最多的类别作为测试集样本X所属的类别

##3.3 随机森林
在自助式抽样的基础上，随机抽取几个属性来构成数据，相当于数据和属性都是随机的

本身来说，决策树是强分类器，只不过在数据随机或者属性随机的情况下，决策树就变成了弱分类器，通过若干个弱分类器的组合，我们就形成了一个可以跟支持向量机相媲美的强分类器

#4 AdaBoost算法
使用装袋算法、随机森林和AdaBoost算法，决策树不需要剪枝的
AdaBoost是一种迭代算法，把弱分类器集合成一个更强的分类器

根据每次训练集中每个样本的分类是否正确，以及上次的总体分类的准确率，来确定每个样本的权重(即改变数据分布)。将修改果权值的新数据集送给下层分类器进行训练，最后将每次得到的分类器融合在一起，作为最后的计策分类器
所以，随机森林和装袋算法是可以并行的，但是AdaBoost算法是串行的

##4.1 AdaBoost思想
- 先通过对N个训练样本的学习得到第一个弱分类器；
- 根据本次分类结果制造一个新的N个的训练样本，通过对这个样本的学习得到第二个弱分类器；
- 根据1，2的分类结果制造一个新的N个的训练样本，通过对这个样本的学习得到第三个弱分类器；
- 最终经过提升的强分类器。即某个数据被分到哪一类都要通过多数表决

##4.2 如何制造新的训练样本
用样本权重模拟样本分布：对于分类错误的样本，加大其对应的权重；而对于分类正确的样本，降低其权重，这样分错的样本就会被突显出来，得到一个新的样本分布

##4.3 如何联合多个弱分类器
使用加权的投票机制代替贫困投票机制。让分类效果好的弱分类器具有较大的权重，而分类效果差的分类器具有较小的权重

##4.4 算法
```
循环迭代，直到累计错误率为0{
	更新样本分布D
	获得当前分布下的最好的弱分类器
	计算最好弱分类器的误差率
	计算最好弱分类器的话语权
}
```
误差率用$\varepsilon$表示，话语权用$\alpha$

给定样本$x_1,x_2,...,x_m$，以及对应的类别$y_1,y_2,...,y_m$,其中$x_i \in X$, $y_i \in Y = \{-1,+1\}$,初始化$D_1(i)=\frac{1}{m}$. 从t=1到T：
- 用当前分布训练一个弱分类器$D_t$；
- 计算弱分类器错误率:$\varepsilon_t$,好的分类器误差率应该小余0.5
- 计算话语权$\alpha_t=\frac{1}{2}ln(\frac{1-\varepsilon_t}{\varepsilon_t})$
- 更新分布:$D_{t+1}(i)=\frac{D_t(i)}{Z_t} * 
\begin{cases}
e^{-\alpha_t} & h_t(x_i) = y_i \\
e^{\alpha_t} & h_t(x_i) \neq y_i
\end{cases}=\frac{D_t(i)e^{\alpha_ty_ih_t(x_i)}}{Z_t}$

最后有：$H(x)=sign(\sum_{t=1}^T\alpha_th_t(x))$

#5 提升
提升是一个机器学习技术，可以用于回归和分类问题，它每一步产生一个弱预测模型，如决策树，并加权累加到总模型中；如果每一步的弱预测模型生成都是依据损失函数的梯度方向，则称之为梯度提升Gradient boosting

梯度提升算法首先给定一个目标损失函数，它的定义域是所有可行的若函数集合(基函数)；提升算法通过迭代的选择一个负梯度方向上的基函数来逐渐逼近局部最小值。

如果一个问题存在弱分类器，则可以通过提升的办法得到强分类器

##5.1 算法
给定输入向量X和输出变量Y组成的若干训练样本$(x_1,y_1),(x_2,y_1),...,(x_n,y_n)$，目标就是找到近似函数$\hat{F}(\overrightarrow{X})$，使得损失函数$L(Y,F(X))$的损失值最小

$L(Y,F(X))$的典型定义以下两种
$$
\begin{cases}
L(Y,F(\overrightarrow{x}))=\frac{1}{2}(y-F(\overrightarrow{x}))^2 \\
L(Y,F(\overrightarrow{x}))=|y-F(\overrightarrow{x})|
\end{cases}
$$
假定最优函数为$F^*(\overrightarrow{x})$，即$F^*(\overrightarrow{x})=argmin_FE_{x,y}[L(y,F(\overrightarrow{x}))]$

假定F(X)是一族基函数$f_i(X)$的加权和$F(\overrightarrow{x})=\sum_{i=1}^M\gamma_if_i(x)+const$

对于式子1来说，$F(\overrightarrow{x})$就是均值，对于式子2来说，$F(\overrightarrow{x})$就是中位数

##5.2 提升算法推导
1. 首先给定常函数$F_0(X)$:$F_0(\overrightarrow{x})=argmin_{\gamma}\sum_{i=1}^nL(y_i,\gamma)$
2. 以贪心的思路扩展得到$F_m(X)$:$F_m(\overrightarrow(x))=F_{m-1}(\overrightarrow{x})+argmin_{f \in H}\sum_{i=1}^nL(y_i, F_{m-1}(\overrightarrow{x_i})+f(\overrightarrow{x_i}))$

对于m=1到M
- 计算伪残差$\gamma_{im}=[\frac{\partial{L(y_i,F(\overrightarrow{x_i}))}}{\partial{F(\overrightarrow{x_i})}}]_{F(\overrightarrow{x})=F_{m-1}(\overrightarrow{x})}$, i=1,2,...,n
- 使用数据$(\overrightarrow{x_i}, \gamma_{im})_{i=1}^n$计算拟合残差的基函数$f_m(x)$
- 计算步长$\gamma_m=argmin_{\gamma}\sum_{i=1}^nL(y_i,F_{m-1}(\overrightarrow{x_i})-\gamma \bullet f_m(\overrightarrow{x_i}))$，这是一个一维优化问题
- 更新模型$F_m(\overrightarrow{x})=F_{m-1}(\overrightarrow{x})-\gamma_mf_m(\overrightarrow{x_i})$

梯度提升的典型基函数就是决策树，尤其是CART决策树。在第m步的梯度提升是根据伪残差数据计算决策树$t_m(X)$。令树$t_m(X)$的叶节点数目为J，即树$t_m(X)$将输入空间划分为J个不相交区域$R_{1m},R_{2m},...,R_{Jm}$，并且决策树$t_m(X)$可以在每个区域中给出某个类型的确定性预测。使用记号$I(X)$，对于输入x，$t_m(X)$为$t_m(\overrightarrow{x})=\sum_{j=1}^Jb_{jm}I(\overrightarrow{x} \in R_{jm})$，其中，$b_{jm}$是样本X在区域$R_{jm}$的预测值

##5.3 二阶导信息
使用一阶导总是会涉及到学习率$\gamma$，我们考虑二阶导

假设有样本X:$\{\overrightarrow{x_1},\overrightarrow{x_2},...,\overrightarrow{x_m}\}$，以及对应的Y个真实值$\{y_1,y_2,....,y_m\}$。目前我们已经找到了t-1个决策树$\{T_1,T_2,...,T_{t-1}\}$，以及对应的t-1个学习率$\{\alpha_1,\alpha_2,...,\alpha_{t-1}\}$。那么对于任意一个样本$\overrightarrow{x_i}$，我们总能算出一个预测值$\hat{y_i}=\alpha_1T_1(x_i)+\alpha_2T_2(x_i)+...+\alpha_{t-1}T_{t-1}(x_i)$。我们使用符号$\hat{y}_{t-1}^{(i)}$来表示使用t-1棵决策树计算出来的第i个样本的预测值，那么我们就有了一组数据$\{(x^{(1)}, \hat{y}_{t-1}^{(1)}), (x^{(2)}, \hat{y}_{t-1}^{(2)}),...,(x^{(m)}, \hat{y}_{t-1}^{(m)})\}$。现在我们要考虑的是怎么计算$T_t(X)$以及$\alpha_t$

目标函数：$J(f_t)=\sum_{i=1}^nL(y_i, \hat{y}_{t-1}^{(i)} + f_t(x_i))+\Omega(f_t)+C$，本身f是未知的

根据Taylor展式:$f(x+\Delta{x})\approx f(x)+f'(x)\Delta{x}+\frac{1}{2}f''(x)\Delta{x}^2$，我们可以看出来$\hat{y}_{t-1}^{(i)}$相当于Taylor展式中的x，$f_t(x_i)$相当于$\Delta{x}$。

令$g_i=\frac{\partial{L(y_i, \hat{y}_{t-1}^{(i)})}}{\partial{\hat{y}_{t-1}^{(i)}}}$,$h_i=\frac{\partial^2{L(y_i, \hat{y}_{t-1}^{(i)})}}{\partial{\hat{y}_{t-1}^{(i)}}}$

由于$\hat{y}_{t-1}^{(i)}$是可以计算出来的，损失函数L是已知的，所以$g_i,h_i$是可以提前计算出来的，所有就有

$J(f_t) \approx \sum_{i=1}^n[L(y_i, \hat{y}_{t-1}^{(i)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

举例说明
![images](images/05/01.png)
这是一个是否喜欢计算机游戏的例子:$f_t(x)=\omega_{q(x)}, \omega \in R^T, q:R^d \{1,2,3,...,T\}$，比如q("boy")=1，因为boy在叶节点1上，也可以是q("boy")=+2，因为叶节点1的权重是+2

##5.4 正则项的定义
决策树的复杂度可考虑叶节点数和叶权值，如使用叶节点总数和叶权值平方和的加权。意思就是说，如果我们觉得一个树很复杂，那么就是说树的叶子节点比较多，并且每个叶子节点的权值比较大。所以我们可以将二阶导数的$\Omega(f_t)$写成这样的形式$\Omega(f_t)=\gamma \bullet T_t + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2$，其中$\gamma,\lambda$是超参数，$T_t$是叶子节点的个数，$\omega$是叶子结点的权重。比如上例中，有3个节点，每个节点的权重为2，0.1，-1,所以$\Omega(f_t)=3\gamma+\frac{1}{2}\lambda(4+0.01+1)$。这就是正则项。这么表示正则项不是唯一的，只不过这么表示比较直观，好推导

对于二阶导，$J(f_t) \approx \sum_{i=1}^n[L(y_i, \hat{y}_{t-1}^{(i)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

$\because L(y_i, \hat{y}_{t-1}^{(i)})$是一个常数，所以可以和最后的C进行合并

$\Rightarrow J(f_t)\approx \sum_{i=1}^n[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

$J(f_t)\approx \sum_{i=1}^n[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\gamma \bullet T_t + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2+C$

